In [1]:
!pip install gdown
!gdown --id '1GAsaI_2KFg9zkQqyTEkl5yl4wA7C39O7'
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip
!unzip glove*.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1GAsaI_2KFg9zkQqyTEkl5yl4wA7C39O7
From (redirected): https://drive.google.com/uc?id=1GAsaI_2KFg9zkQqyTEkl5yl4wA7C39O7&confirm=t&uuid=71ff0f37-fdc7-4b7e-8e9a-af90d333f8d4
To: /content/sentiment140.csv
100% 203M/203M [00:02<00:00, 80.9MB/s]
--2024-06-02 19:02:06--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.49, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/6471382cdd837544bf3ac72497a38715e845897d265b2b424b4761832009c837?response-content-disposition=attachment%3B

In [2]:
import csv
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, random_split
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
with open('sentiment140.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    twits = []
    y_arr = []
    for row in reader:
        text = row['text']
        sentiment = int(row['sentiment'])
        twits.append(text)
        if sentiment == 0:
           y_arr.append([0,1])
        elif sentiment == 4:
           y_arr.append([1,0])
        else:
           print('shouldnt get here')

In [4]:
url_pattern = r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
mention_pattern = r'@([A-Za-z0-9_]+)'
hasgtag_pattern = r'#\w+'
cleaned_twits = []
for text in twits:
    cleaned_text = re.sub(url_pattern, "<url>", text)
    cleaned_text = re.sub(mention_pattern, "<mention>", cleaned_text)
    cleaned_text = re.sub(hasgtag_pattern, "<hasgtag>", cleaned_text)
    cleaned_twits.append(cleaned_text)

In [5]:
texts_tokens = []
lemmatizer = WordNetLemmatizer()
for text in cleaned_twits:
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in string.punctuation]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    texts_tokens.append(filtered_tokens)

In [7]:
for i in range(20):
    print(cleaned_twits[i])

<mention> <url> - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
<mention> I dived many times for the ball. Managed to save 50%  The rest go out of bounds
my whole body feels itchy and like its on fire 
<mention> no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
<mention> not the whole crew 
Need a hug 
<mention> hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?
<mention> nope they didn't have it 
<mention> que me muera ? 
spring break in plain city... it's snowing 
I just re-pierced my ears 
<mention> I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . .
<mention> It it counts, idk why I did either. you never talk to me anymore 
<mention> i would've been the first, but i didn't have a gun.    not really though, zac snyder's just a do

In [6]:
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

In [8]:
glove_file_path = './glove.6B.50d.txt'
word_vectors = load_glove_vectors(glove_file_path)

In [9]:
def get_vectors_of_given_word(word):
    if word in word_vectors:
       return word_vectors[word]
    else:
       return None

In [10]:
word = 'feat'
print(get_vectors_of_given_word(word))

[-0.4623     0.4685    -0.059266   0.31502   -0.20935    0.27924
 -0.02232    0.45211    0.41786    1.382      0.065629   0.0095435
 -1.1704    -0.26955    0.47909   -0.60971    1.1074     0.088239
 -0.73538   -0.10438   -0.68514    0.10573    0.26877   -0.99686
  0.8547    -0.10986   -1.3006    -0.41911    0.45025   -0.29343
  1.329      0.14538    0.028881  -0.10911    0.57076    0.19576
  0.7261     0.8676    -0.88164   -0.65089   -0.61051   -0.78066
 -0.50536   -0.55755   -0.21071   -0.39848    0.22182   -0.1319
 -0.40512    0.17668  ]


In [26]:
def get_vectors_of_given_sentence(sentence):
    vec = np.zeros(50)
    tokens = nltk.word_tokenize(sentence)
    ntokens = [token for token in tokens if token not in string.punctuation]
    for t in ntokens:
        arr = get_vectors_of_given_word(t)
        arrlist = []
        if not arr is None:
           arrlist.append(arr)
    for i in range(50):
        count = 0
        for arr in arrlist:
            count += arr[i]
        vec[i] = count/len(arrlist)

    return vec

In [27]:
sentence = 'I dont know what youre talking about'
get_vectors_of_given_sentence(sentence)

array([ 0.89466   ,  0.36603999,  0.37588   , -0.41817999,  0.58462   ,
        0.18594   , -0.41907001, -0.46621001, -0.54903001,  0.02477   ,
       -0.90815997, -0.48271   , -0.050742  , -0.74039   ,  1.43770003,
       -0.01974   , -0.2384    ,  0.43154001, -0.66119999, -0.41275001,
        0.25475001,  0.93497998,  0.81404001, -0.17296   ,  0.61295998,
       -1.84749997, -0.27616   ,  0.27700999,  0.42346999, -0.11599   ,
        3.6243    ,  0.12306   , -0.023526  , -0.24843   , -0.22375999,
       -0.53941   , -0.62444001, -0.27711001,  0.49406001,  0.020234  ,
       -0.23459999,  0.44512001,  0.53397   ,  0.66654003, -0.093662  ,
       -0.035203  , -0.064194  ,  0.55997998, -0.66592997,  0.12177   ])

In [28]:
del twits
del cleaned_twits
del cleaned_text

In [29]:
x_arr = []
max_len = 0
for text in texts_tokens:
    x = np.empty((0,50))
    for word in text:
        vectors = get_vectors_of_given_word(word)
        if not vectors is None:
            vectors_np = np.array(vectors)
            x = np.vstack([x, vectors_np])
    if max_len < x.shape[0]:
         max_len = x.shape[0]
    if x.shape[0] != 0:
       x_arr.append(x)

In [30]:
del texts_tokens

In [31]:
class CustomDataset(Dataset):
    def __init__(self, x_list, y_list):
        self.samples = []
        while x_list:
            x = x_list.pop()
            y = y_list.pop()
            x_tensor = torch.tensor(x,dtype = torch.float32).cuda()
            y_tensor = torch.tensor(y,dtype = torch.float32).cuda()
            self.samples.append((x_tensor,y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [32]:
dataset = CustomDataset(x_arr,y_arr)

In [33]:
del x_arr
del y_arr

In [34]:
train_dataset, test_dataset = random_split(dataset, [1270047, 317511])

In [35]:
del dataset

In [36]:
def get_maxlen(batch):
    maxlen = 0
    for tensr in batch:
        if tensr[0].shape[0] > maxlen :
           maxlen = tensr[0].shape[0]
    return maxlen

In [37]:
def collate_fn(batch):
    xs = []
    ys = []
    max_len = get_maxlen(batch)
    for tensr in batch:
        y_tensor = tensr[1]
        x_tensor = tensr[0]
        pad_width = max_len - x_tensor.shape[0]
        leftpad_width = pad_width // 2
        rightpad_width = pad_width - leftpad_width
        p1d = (0,0,leftpad_width,rightpad_width)
        paded_x_tensor = F.pad(tensr[0],p1d,"constant",0)
        xs.append(paded_x_tensor)
        ys.append(y_tensor)
    X = torch.stack(xs,dim=0)
    Y = torch.stack(ys,dim=0)
    return [X,Y]

In [38]:
train_loader = DataLoader(train_dataset, batch_size=512,collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1,collate_fn=collate_fn, shuffle=False)

In [39]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.lstm = nn.LSTM(50, 20, 1,batch_first = True)
        self.fc1 = nn.Linear(20, 2)

    def forward(self, x):
        h0 = torch.zeros(1, 32, 20)
        c0 = torch.zeros(1, 32, 20)
        output, _ = self.lstm(x)
        x = output[:,-1,:]
        x = nn.functional.softmax(self.fc1(x))
        return x

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [41]:
model.train()
num_epochs = 5
for epoch in range(num_epochs):
    for sente, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(sente)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #if (i+1) % 100 == 0:
            #print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')
    print(loss.item())

print('Training complete.')

<ipython-input-39-e13706f5eece>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc1(x))


0.5498431921005249
0.5524373054504395
0.5554583668708801
0.4906899034976959
0.5367427468299866
Training complete.


In [42]:
predictions = []
true_labels = []
i = 0

for inputs, labels in test_loader:
    outputs = model(inputs)
    predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    l1 = torch.argmax(labels, dim=1).cpu().numpy()
    true_labels.extend(l1)

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

precision = precision_score(true_labels, predictions, average='weighted')
print("Precision:", precision)

recall = recall_score(true_labels, predictions, average='weighted')
print("Recall:", recall)

f1 = f1_score(true_labels, predictions, average='weighted')
print("F1 Score:", f1)

<ipython-input-39-e13706f5eece>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc1(x))


Accuracy: 0.7255780114704687
Precision: 0.7348572388589042
Recall: 0.7255780114704687
F1 Score: 0.7230525353891081


In [43]:
class NN1(nn.Module):
    def __init__(self):
        super(NN1, self).__init__()
        self.lstm = nn.GRU(50, 20, 1,batch_first = True)
        self.fc1 = nn.Linear(20, 2)

    def forward(self, x):
        h0 = torch.zeros(1, 32, 20)
        c0 = torch.zeros(1, 32, 20)
        output, _ = self.lstm(x)
        x = output[:,-1,:]
        x = nn.functional.softmax(self.fc1(x))
        return x

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model1 = NN1()
model1.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [45]:
model1.train()
num_epochs = 5
for epoch in range(num_epochs):
    for sente, labels in train_loader:
        optimizer.zero_grad()
        outputs = model1(sente)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #if (i+1) % 100 == 0:
            #print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')
    print(loss.item())

print('Training complete.')

<ipython-input-43-a3e62b2f3025>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc1(x))


0.5815762281417847
0.5288255214691162
0.526593804359436
0.5288028717041016
0.5261754989624023
Training complete.


In [46]:
predictions = []
true_labels = []
i = 0

for inputs, labels in test_loader:
    outputs = model1(inputs)
    predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    l1 = torch.argmax(labels, dim=1).cpu().numpy()
    true_labels.extend(l1)

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

precision = precision_score(true_labels, predictions, average='weighted')
print("Precision:", precision)

recall = recall_score(true_labels, predictions, average='weighted')
print("Recall:", recall)

f1 = f1_score(true_labels, predictions, average='weighted')
print("F1 Score:", f1)

<ipython-input-43-a3e62b2f3025>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc1(x))


Accuracy: 0.7463772908655133
Precision: 0.7481629196907097
Recall: 0.7463772908655133
F1 Score: 0.7459987769293417
